### 라이브러리 불러오기

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from flask import Flask, render_template
import pandas as pd

### 네이버에서 야구 정보를 가져오기

In [1]:
record_list=[]

class Record():
    def __init__(self,tuple10):
        self.name=tuple10[0]
        self.ranking=tuple10[1]
        self.year=tuple10[2]
        self.plays=tuple10[3]
        self.wins=tuple10[4]
        self.losses=tuple10[5]
        self.win_rate=tuple10[6]
        self.draws=tuple10[7]
        self.obp=tuple10[8]
        self.slg=tuple10[9] 
        
    
def get_info(ranking,team_list,i,year):
    name=team_list[i+1].text
    ranking=ranking
    year=year
    plays=int(team_list[i+2].text)
    wins=int(team_list[i+3].text)
    losses=int(team_list[i+4].text)
    win_rate=wins/(plays-losses)
    draws=int(team_list[i+5].text)
    obp=float(team_list[i+8].text)
    slg=float(team_list[i+9].text)
    return name,ranking,year,plays,wins,losses,win_rate,draws,obp,slg

    
chrome_driver_path = 'etc/chromedriver.exe' 

with webdriver.Chrome(executable_path=chrome_driver_path) as driver:
    for i in range(1991,2024):
        adress="https://sports.news.naver.com/kbaseball/record/index?category=kbo&year="+str(i)
        
        driver.get(adress)

        # 페이지에서 롯데 정보 긁기
        tr=driver.find_element(By.CLASS_NAME, "tbl_box")
        team_list=tr.find_elements(By.TAG_NAME, 'span')

        year=driver.find_element(By.XPATH,'//*[@id="_currentYearButton"]/em').text
        year=int(year.replace('선택된 시즌\n',''))
        
        checking_start, checking_end = 12, 100
        if year in (2013,2014):
            checking_end+=11
        elif year > 2014:
            checking_end+=22
        ranking=0
        for i in range(checking_start,checking_end,11):
            ranking+=1
            record=Record( 
                get_info(ranking,team_list,i,year)
            )
            record_list.append(record)

C:\Users\user\AppData\Local\Temp\ipykernel_5688\2846975370.py:37: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with webdriver.Chrome(executable_path=chrome_driver_path) as driver:


### csv파일로 저장

In [6]:
df = pd.DataFrame([record.__dict__ for record in record_list])

# CSV 파일에 데이터 저장
filename = 'Playball_1991_2023.csv'
df.to_csv(filename, index=False)

### csv파일 불러오기

In [4]:
df=pd.read_csv("csv/Playball_1991_2023.csv")
df

,name,ranking,year,plays,wins,losses,draws,obp,slg,win_rate
0,해태,1,1991,126,79,42,5,0.350,0.418,0.653
1,빙그레,2,1991,126,72,49,5,0.353,0.433,0.595
2,삼성,3,1991,126,70,55,1,0.357,0.405,0.560
3,롯데,4,1991,126,61,62,3,0.339,0.378,0.496
4,태평양,5,1991,126,55,69,2,0.313,0.337,0.444
...,...,...,...,...,...,...,...,...,...,...
279,KIA,6,2023,33,16,17,0,0.330,0.345,0.485
280,키움,7,2023,39,17,22,0,0.328,0.354,0.436
281,삼성,8,2023,35,15,20,0,0.322,0.352,0.429
282,한화,9,2023,37,13,22,2,0.308,0.314,0.371


In [11]:
group_df=df[['name','wins','losses']].groupby('name').sum()
group_df.loc['롯데']

wins      1973
losses    2211
Name: 롯데, dtype: int64

### Flask 웹 구동

In [20]:


app = Flask(__name__)

@app.route('/')
def home():
    # 야구 정보 데이터프레임
    records = df.to_dict('records')
    
    return render_template('index.html', records=records)

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [19/May/2023 02:47:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2023 02:47:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2023 02:47:59] "GET / HTTP/1.1" 200 -
